In [1]:

!pip install transformers[sentencepiece] --quiet
!pip install torch --quiet
!pip install pandas openpyxl psutil python-dotenv --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) <= 3)

!ls /kaggle/working/

# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    !mkdir -p /kaggle/working/analysis/fine_tuned_model/UBC-NLP_AraT5v2-base-1024/final/
    !cp -r /kaggle/input/ubc-nlp-rat5v2-base-1024-model/UBC-NLP_AraT5v2-base-1024/final/* /kaggle/working/analysis/fine_tuned_model/UBC-NLP_AraT5v2-base-1024/final/
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")

# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

__notebook__.ipynb
✅ '/kaggle/working' is empty. copying project strcuture into it ...
/kaggle/working
analysis   data        __init__.py  __notebook__.ipynb	run_pipeline.py
config.py  evaluation  models	    prompts		utils


In [4]:
!ls -R /kaggle/working/analysis/fine_tuned_model/UBC-NLP_AraT5v2-base-1024

/kaggle/working/analysis/fine_tuned_model/UBC-NLP_AraT5v2-base-1024:
final

/kaggle/working/analysis/fine_tuned_model/UBC-NLP_AraT5v2-base-1024/final:
added_tokens.json	model.safetensors	 tokenizer_config.json
config.json		special_tokens_map.json  training_args.bin
generation_config.json	spiece.model


In [5]:
MODEL_NAME = "UBC-NLP/AraT5v2-base-1024" #"agemagician_mlong-t5-tglobal-large" #"facebook_mbart-large-50-many-to-many-mmt" # "UBC-NLP_AraT5-base" #"google/mt5-small"

# !unzip -o /content/drive/MyDrive/AraGenEval2025/models/fine_tuned_model/model_weights/{model_name}/final_model_weights.zip -d /content/drive/MyDrive/AraGenEval2025/models/fine_tuned_model/model_weights/{model_name}/

In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [7]:

import os
import json
import dotenv
import torch
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [8]:
# project_dir = "/content/drive/MyDrive/AraGenEval2025"
# os.chdir(project_dir)
# print(f"✅ Current directory: {os.getcwd()}")


In [9]:

import sys
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from config import Config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")


✅ Using device: cpu


In [10]:

def safe_model_name(model_name):
    return model_name.replace("/", "_")

def create_run_dir(base_dir, model_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(base_dir, safe_model_name(model_name), f"run_{timestamp}")
    os.makedirs(output_dir, exist_ok=True)
    return output_dir


In [11]:

def load_model(model_name):
    secure_name = safe_model_name(model_name)
    print(f"Loading model: {secure_name}")
    model_path = os.path.join("models/fine_tuned_model/model_weights", secure_name, "final")
    print(f" loading fine tuned model from {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    if "mbart" in model_name.lower():
        tokenizer.src_lang = "ar_AR"
        tokenizer.tgt_lang = "ar_AR"

    model.to(device)
    model.eval()
    return tokenizer, model


In [12]:

def run_evaluation():
  pass
    # os.system("python utils/prepare_metrics_log.py")
    # os.system("python evaluation/evaluate.py")

def archive_results(run_dir, files_to_archive):
    for fpath in files_to_archive:
        if os.path.exists(fpath):
            dest = os.path.join(run_dir, os.path.basename(fpath))
            os.system(f"cp {fpath} {dest}")


In [13]:
from tqdm import tqdm
import pandas as pd
import torch
import gc
import json

def generate_predictions(tokenizer, model, validation_path, output_log_path, batch_size=8):
    # Load validation data
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

    # Ensure correct special token settings
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    assert model.config.eos_token_id is not None, "EOS token must be defined"

    records = []

    for start in tqdm(range(0, len(df), batch_size)):
        batch_texts = df["input_text"].iloc[start:start+batch_size].tolist()

        # Tokenize full input without truncation
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=False  # ensures full input preservation :contentReference[oaicite:1]{index=1}
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            gen_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                num_beams=4,           # beam search for coherent output :contentReference[oaicite:2]{index=2}
                early_stopping=True,   # stops when beams finish naturally
                do_sample=False,        # deterministic decoding
                max_length=4096  # Set this one line only!
            )

        outputs = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

        for idx, output_text in enumerate(outputs):
            row = df.iloc[start + idx]
            records.append({
                "id": int(row["id"]),
                "author": str(row["author"]),
                "output": output_text,
                "ground_truth": str(row["text_in_author_style"])
            })

        # Clean up memory
        torch.cuda.empty_cache()
        gc.collect()

    with open(output_log_path, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"✅ Finished generation for {len(records)} samples.")


In [14]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

30

In [15]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [16]:

# model_name = "facebook/mbart-large-50-many-to-many-mmt" #"UBC-NLP_AraT5-base" #"google/mt5-small"
validation_file = Config.VAL_FILE

log_file = f"models/fine_tuned_model/results/{MODEL_NAME}/generation_log.json"   #Config.LOG_FILE
output_base_dir =  'analysis/fine_tuned_model' #Config.FINE_TUNED_ANALYSIS_DIR

print(log_file)
print(output_base_dir)


models/fine_tuned_model/results/UBC-NLP/AraT5v2-base-1024/generation_log.json
analysis/fine_tuned_model


In [17]:
from tqdm import tqdm
import pandas as pd
import torch
import gc
import json

def generate_sample_predictions(tokenizer, model, validation_path, sample_count=3):
    # Load validation data
    df = pd.read_excel(validation_path)
    df["input_text"] = df["author"].astype(str) + " [SEP] " + df["text_in_msa"].astype(str)

    # Use only first few samples
    df = df.iloc[:sample_count].copy()

    # Ensure correct special token settings
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    assert model.config.eos_token_id is not None, "EOS token must be defined"

    print(f"\n🔍 Testing on {sample_count} samples...\n")
    records = []

    for i in tqdm(range(len(df))):
        input_text = df.iloc[i]["input_text"]
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            padding=True,
            truncation=True,  # Full input preserved
            max_length=1280
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            gen_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                # num_beams=1,
                # early_stopping=True,
                # do_sample=False,
                max_length=3000  # Important to allow full output
            )

        output_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)

        record = {
            "id": int(df.iloc[i]["id"]),
            "author": str(df.iloc[i]["author"]),
            "input": str(df.iloc[i]["text_in_msa"]),
            "output": output_text,
            "ground_truth": str(df.iloc[i]["text_in_author_style"])
        }

        records.append(record)

        # Print result for inspection
        print(f"\n📝 Sample {i+1}")
        print(f"Author: {record['author']}")
        print(f"Input:\n{record['input'][:500]}{'...' if len(record['input']) > 500 else ''}")
        print(f"\n🔁 Output:\n{record['output']}")
        print(f"\n✅ Ground Truth:\n{record['ground_truth'][:500]}{'...' if len(record['ground_truth']) > 500 else ''}")
        print("=" * 100)

        # Memory cleanup
        torch.cuda.empty_cache()
        gc.collect()

    return records


    print(f"\n✅ Test finished for {num_samples} samples.\n")


In [18]:
# tokenizer, model = load_model(MODEL_NAME)
# generate_sample_predictions(tokenizer, model, validation_file, sample_count=3)


In [19]:
tokenizer, model = load_model(MODEL_NAME)

os.makedirs(os.path.dirname(log_file) or ".", exist_ok=True)
generate_predictions(tokenizer, model, validation_file, log_file, batch_size=4)

# labeled_log = Config.LABELED_LOG
# metrics_log = Config.PER_AUTHOR_METRICS_LOG
# run_evaluation()

output_dir = create_run_dir(output_base_dir, MODEL_NAME)
archive_results(output_dir, [log_file]) # , labeled_log, metrics_log
print(log_file)

print(f"\n✅ Pipeline completed. Results saved to: {output_dir}")

Loading model: UBC-NLP_AraT5v2-base-1024
 loading fine tuned model from models/fine_tuned_model/model_weights/UBC-NLP_AraT5v2-base-1024/final


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'models/fine_tuned_model/model_weights/UBC-NLP_AraT5v2-base-1024/final'. Use `repo_type` argument if needed.